In [4]:
from datafusion import SessionContext
import polars as pl

pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(-1)

file = "data/foo.parquet"
ctx = SessionContext()
ctx.register_parquet("t", file)
df = ctx.sql("select * from t limit 10")
# df = ctx.sql("select * exclude(name) \
#              from t \
#              order by id")
print(df.schema())
df = df.to_polars()
df

id: int32 not null
name: string


id,name
i32,str
1,"""foo"""
2,"""bar"""
3,"""baz"""


In [5]:
import polars as pl

# pl.Config.restore_defaults()
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(-1)

file = "data/foo.parquet"
df = pl.scan_parquet(file)
# df.schema
context = pl.SQLContext(frame=df)
query = """
    select *
    from frame
    limit 10
    """
context.execute(query).collect()

id,name
i32,str
1,"""foo"""
2,"""bar"""
3,"""baz"""


In [7]:
import polars as pl

pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(-1)

file = "./data/foo2.parquet"
df = pl.scan_parquet(file)
df = df.with_columns([
    pl.col('metadata').str.json_path_match(r"$.name").alias('name'),
    pl.col('metadata').str.json_path_match(r"$.data").alias('data'),
])
context = pl.SQLContext(frame=df)
query = """
    select *
    from frame
    limit 10
    """
context.execute(query).collect()

id,metadata,name,data
i32,str,str,str
1,"""{""data"":42,""name"":""foo""}""","""foo""","""42"""
3,"""{""data"":44,""name"":""baz""}""","""baz""","""44"""
2,"""{""data"":43,""name"":""bar""}""","""bar""","""43"""


In [10]:
import duckdb

duckdb.sql(
    """
    select *
    from 'data/foo.parquet'
    limit 10
    """
)

┌───────┬─────────┐
│  id   │  name   │
│ int32 │ varchar │
├───────┼─────────┤
│     1 │ foo     │
│     2 │ bar     │
│     3 │ baz     │
└───────┴─────────┘

In [11]:
import pyarrow.parquet as pq

file = "./data/foo.parquet"
df = pq.read_table(file)
df.to_pandas().head(5)

,id,name
0,1,foo
1,2,bar
2,3,baz
